In [1]:
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH17-LangGraph-Use-Cases")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH17-LangGraph-Use-Cases


# 기본 PDF 기반 Retrieval Chain 생성


In [2]:
from rag.pdf import PDFRetrievalChain

# PDF 문서를 로드
pdf = PDFRetrievalChain(["data/SPRI_AI_Brief_2023년12월호_F.pdf"]).create_chain()

# retriever 와 chain을 생성
pdf_retriever = pdf.retriever
pdf_chain = pdf.chain

# 검색된 문서의 관련성 평가 (Question-Retrieval Evaluation)


In [3]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_teddynote.models import get_model_name, LLMs
from pydantic import BaseModel, Field

# 모델 이름 가져오기
MODEL_NAME = get_model_name(LLMs.GPT4)


# 검색된 문서의 관련성 여부를 이진 점수로 평가하는 데이터 모델
class GradeDocuments(BaseModel):
    """A binary score to determine the relevance of the retrieved document."""

    # 문서가 질문과 관련이 있는지 여부를 'yes' 또는 'no'로 나타내는 필드
    binary_score: str = Field(
        description="Documents are relevant to the question, 'yes' or 'no'"
    )


# LLM 초기화
llm = ChatOpenAI(model=MODEL_NAME, temperature=0)

# GradeDocuments 데이터 모델을 사용하여 구조화된 출력을 생성하는 LLM
structured_llm_grader = llm.with_structured_output(GradeDocuments)

# 시스템 프롬프트 정의
system = """You are a grader assessing relevance of a retrieved document to a user question. \n
    If the document contains keyword(s) or semantic meaning related to the question, grade it as relevant. \n
    Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question."""

# 채팅 프롬프트 템플릿 생성
grade_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Retrieved document: \n\n {document} \n\n User question: {question}"),
    ]
)

# Retrieval 평가기 초기화
retrieval_grader = grade_prompt | structured_llm_grader

In [4]:
# 질문 정의
question = "삼성전자가 개발한 생성AI 에 대해 설명하세요."

# 문서 검색
docs = pdf_retriever.invoke(question)

# 검색된 문서 중 1번 index 문서의 페이지 내용을 추출
doc_txt = docs[1].page_content

# 검색된 문서와 질문을 사용하여 관련성 평가를 실행하고 결과 출력
print(retrieval_grader.invoke({"question": question, "document": doc_txt}))

binary_score='yes'


# 답변 생성 체인


In [5]:
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI


# LangChain Hub에서 RAG 프롬프트를 가져와 사용
prompt = hub.pull("teddynote/rag-prompt")

# LLM 초기화
llm = ChatOpenAI(model_name=MODEL_NAME, temperature=0)


# 문서 포맷팅
def format_docs(docs):
    return "\n\n".join(
        [
            f'<document><content>{doc.page_content}</content><source>{doc.metadata["source"]}</source><page>{doc.metadata["page"]+1}</page></document>'
            for doc in docs
        ]
    )


# 체인 생성
rag_chain = prompt | llm | StrOutputParser()


# 체인 실행 및 결과 출력
generation = rag_chain.invoke({"context": format_docs(docs), "question": question})
print(generation)

삼성전자가 개발한 생성 AI '삼성 가우스'는 온디바이스에서 작동하며, 언어, 코드, 이미지의 3개 모델로 구성되어 있습니다. 이 모델은 외부로 사용자 정보가 유출될 위험이 없으며, 다양한 제품에 단계적으로 탑재될 계획입니다. 삼성 가우스는 안전한 데이터를 통해 학습되었고, 최적화된 크기의 모델 선택이 가능합니다.

**Source**
- data/SPRI_AI_Brief_2023년12월호_F.pdf (page 13)


# 쿼리 재작성(Question Re-writer)


In [6]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

# LLM 설정
llm = ChatOpenAI(model=MODEL_NAME, temperature=0)

# Query Rewrite 시스템 프롬프트
system = """You a question re-writer that converts an input question to a better version that is optimized
for web search. Look at the input and try to reason about the underlying semantic intent / meaning."""

# 프롬프트 정의
re_write_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        (
            "human",
            "Here is the initial question: \n\n {question} \n Formulate an improved question.",
        ),
    ]
)

# Question Re-writer 체인 초기화
question_rewriter = re_write_prompt | llm | StrOutputParser()

In [7]:
# 실행 및 결과 확인
print(f'[원본 질문]: "{question}"')
print("[쿼리 재작성]:", question_rewriter.invoke({"question": question}))

[원본 질문]: "삼성전자가 개발한 생성AI 에 대해 설명하세요."
[쿼리 재작성]: "삼성전자가 개발한 생성 AI의 특징과 기능에 대해 자세히 설명해 주세요."


# 웹 검색 도구


In [8]:
# 웹 검색 도구 초기화
from langchain_teddynote.tools.tavily import TavilySearch

# 최대 검색 결과를 3으로 설정
web_search_tool = TavilySearch(max_results=3)


In [9]:
# 웹 검색 도구 실행
results = web_search_tool.invoke({"query": question})
print(results)

[{'title': '삼성전자, 생성 Ai 모델 \'삼성 가우스\' 공개...언어‧코드‧이미지 3개 모델, "생산성 향상과 새로운 사용자 경험 제공할 것!"', 'url': 'https://www.aitimes.kr/news/articleView.html?idxno=29318', 'content': '삼성전자, 생성 AI 모델 \'삼성 가우스\' 공개...언어‧코드‧이미지 3개 모델, "생산성 향상과 새로운 사용자 경험 제공할 것!" < 플랫폼 < AI Tech < 기사본문 - 인공지능신문 삼성전자, 생성 AI 모델 \'삼성 가우스\' 공개...언어‧코드‧이미지 3개 모델, "생산성 향상과 새로운 사용자 경험 제공할 것!" 삼성전자, 생성 AI 모델 \'삼성 가우스\' 공개...언어‧코드‧이미지 3개 모델, "생산성 향상과 새로운 사용자 경험 제공할 것!" 삼성전자가 8일, 삼성전자 서울R&D캠퍼스에서 열린 \'삼성 AI 포럼 2023\'을 통해\xa0생성 AI 모델 \'삼성 가우스(Samsung Gauss)\'를 최초 공개했다. 또 "삼성전자는 AI 기술 개발뿐만 아니라 AI 활용시 보안 및 안전성을 확보하기 위해 \'AI 윤리 원칙\'을 정해 이를 준수하기 위해 노력하고 있다"며, "삼성전자는 생성형 AI에 대한 지속적 연구를 통해 소비자의 경험 가치를 높여 나갈 계획"이라고 강조했다.', 'score': 0.7555112, 'raw_content': '삼성전자, 생성 AI 모델 \'삼성 가우스\' 공개...언어‧코드‧이미지 3개 모델, "생산성 향상과 새로운 사용자 경험 제공할 것!" < 플랫폼 < AI Tech < 기사본문 - 인공지능신문\n주요서비스 바로가기 본문 바로가기 매체정보 바로가기 로그인 바로가기 기사검색 바로가기 전체서비스 바로가기\n상단영역\n\n취재의뢰\n기사제보\n뉴스레터신청\n행사문의\n\n후원·협찬문의\n\n\n로그인\n\n회원가입\n모바일웹\n\n\n\n전체메뉴 버튼\n\nAI Tech\nMachine Learnig\nDeep L

# 상태(State)

In [10]:
from typing import Annotated, List
from typing_extensions import TypedDict


# 상태 정의
class GraphState(TypedDict):
    question: Annotated[str, "The question to answer"]
    generation: Annotated[str, "The generation from the LLM"]
    web_search: Annotated[str, "Whether to add search"]
    documents: Annotated[List[str], "The documents retrieved"]

# 노드

In [11]:
from langchain.schema import Document


# 문서 검색 노드
def retrieve(state: GraphState):
    print("\n==== RETRIEVE ====\n")
    question = state["question"]

    # 문서 검색 수행
    documents = pdf_retriever.invoke(question)
    return {"documents": documents}


# 답변 생성 노드
def generate(state: GraphState):
    print("\n==== GENERATE ====\n")
    question = state["question"]
    documents = state["documents"]

    # RAG를 사용한 답변 생성
    generation = rag_chain.invoke({"context": documents, "question": question})
    return {"generation": generation}


# 문서 평가 노드
def grade_documents(state: GraphState):
    print("\n==== [CHECK DOCUMENT RELEVANCE TO QUESTION] ====\n")
    question = state["question"]
    documents = state["documents"]

    # 필터링된 문서
    filtered_docs = []
    relevant_doc_count = 0

    for d in documents:
        # Question-Document 의 관련성 평가
        score = retrieval_grader.invoke(
            {"question": question, "document": d.page_content}
        )
        grade = score.binary_score

        if grade == "yes":
            print("==== [GRADE: DOCUMENT RELEVANT] ====")
            # 관련 있는 문서를 filtered_docs 에 추가
            filtered_docs.append(d)
            relevant_doc_count += 1
        else:
            print("==== [GRADE: DOCUMENT NOT RELEVANT] ====")
            continue

    # 관련 문서가 없으면 웹 검색 수행
    web_search = "Yes" if relevant_doc_count == 0 else "No"
    return {"documents": filtered_docs, "web_search": web_search}


# 쿼리 재작성 노드
def query_rewrite(state: GraphState):
    print("\n==== [REWRITE QUERY] ====\n")
    question = state["question"]

    # 질문 재작성
    better_question = question_rewriter.invoke({"question": question})
    return {"question": better_question}


# 웹 검색 노드
def web_search(state: GraphState):
    print("\n==== [WEB SEARCH] ====\n")
    question = state["question"]
    documents = state["documents"]

    # 웹 검색 수행
    docs = web_search_tool.invoke({"query": question})
    # 검색 결과를 문서 형식으로 변환
    web_results = "\n".join([d["content"] for d in docs])
    web_results = Document(page_content=web_results)
    documents.append(web_results)

    return {"documents": documents}

# 조건부 엣지에 활용할 함수

In [12]:
def decide_to_generate(state: GraphState):
    # 평가된 문서를 기반으로 다음 단계 결정
    print("==== [ASSESS GRADED DOCUMENTS] ====")
    # 웹 검색 필요 여부
    web_search = state["web_search"]

    if web_search == "Yes":
        # 웹 검색으로 정보 보강이 필요한 경우
        print(
            "==== [DECISION: ALL DOCUMENTS ARE NOT RELEVANT TO QUESTION, QUERY REWRITE] ===="
        )
        # 쿼리 재작성 노드로 라우팅
        return "query_rewrite"
    else:
        # 관련 문서가 존재하므로 답변 생성 단계(generate) 로 진행
        print("==== [DECISION: GENERATE] ====")
        return "generate"

# 그래프 생성

In [13]:
from langgraph.graph import END, StateGraph, START

# 그래프 상태 초기화
workflow = StateGraph(GraphState)

# 노드 정의
workflow.add_node("retrieve", retrieve)
workflow.add_node("grade_documents", grade_documents)
workflow.add_node("generate", generate)
workflow.add_node("query_rewrite", query_rewrite)
workflow.add_node("web_search_node", web_search)

# 엣지 연결
workflow.add_edge(START, "retrieve")
workflow.add_edge("retrieve", "grade_documents")

# 문서 평가 노드에서 조건부 엣지 추가
workflow.add_conditional_edges(
    "grade_documents",
    decide_to_generate,
    {
        "query_rewrite": "query_rewrite",
        "generate": "generate",
    },
)

# 엣지 연결
workflow.add_edge("query_rewrite", "web_search_node")
workflow.add_edge("web_search_node", "generate")
workflow.add_edge("generate", END)

# 그래프 컴파일
app = workflow.compile()

# 그래프 실행

In [14]:
from langchain_core.runnables import RunnableConfig
from langchain_teddynote.messages import stream_graph, invoke_graph, random_uuid

# config 설정(재귀 최대 횟수, thread_id)
config = RunnableConfig(recursion_limit=20, configurable={"thread_id": random_uuid()})

# 질문 입력
inputs = {
    "question": "삼성전자가 개발한 생성형 AI 의 이름은?",
}

# 스트리밍 형식으로 그래프 실행
stream_graph(
    app,
    inputs,
    config,
    ["retrieve", "grade_documents", "query_rewrite", "web_search_node", "generate"],
)


==== RETRIEVE ====


==== [CHECK DOCUMENT RELEVANCE TO QUESTION] ====


🔄 Node: grade_documents 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
{"binary_score":"yes"}==== [GRADE: DOCUMENT RELEVANT] ====
{"binary_score":"yes"}==== [GRADE: DOCUMENT RELEVANT] ====
{"binary_score":"yes"}==== [GRADE: DOCUMENT RELEVANT] ====
{"binary_score":"yes"}==== [GRADE: DOCUMENT RELEVANT] ====
{"binary_score":"no"}==== [GRADE: DOCUMENT NOT RELEVANT] ====
{"binary_score":"no"}==== [GRADE: DOCUMENT NOT RELEVANT] ====
{"binary_score":"yes"}==== [GRADE: DOCUMENT RELEVANT] ====
{"binary_score":"yes"}==== [GRADE: DOCUMENT RELEVANT] ====
{"binary_score":"no"}==== [GRADE: DOCUMENT NOT RELEVANT] ====
{"binary_score":"no"}==== [GRADE: DOCUMENT NOT RELEVANT] ====
==== [ASSESS GRADED DOCUMENTS] ====
==== [DECISION: GENERATE] ====

==== GENERATE ====


🔄 Node: generate 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
삼성전자가 개발한 생성형 AI의 이름은 '삼성 가우스'입니다.

**Source**
- data/SPRI_AI_Brief_2023년12월호_F.pdf (page

In [15]:
# config 설정(재귀 최대 횟수, thread_id)
config = RunnableConfig(recursion_limit=20, configurable={"thread_id": random_uuid()})

# 질문 입력
inputs = {
    "question": "2024년 노벨문학상 수상자의 이름은?",
}

In [16]:
# 그래프 실행
invoke_graph(app, inputs, config)


==== RETRIEVE ====


🔄 Node: retrieve 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
page_content='CES 2024
이번 전시에는 500곳 이상의 한국기업 참가 예정
기간 장소 홈페이지
2024.1.9~12 미국, 라스베가스 https://www.ces.tech/
- 머신러닝 및 응용에 관한 국제 컨퍼런스(AIMLA 2024)는
인공지능 및 머신러닝의 이론, 방법론 및 실용적 접근에 관한
지식과 최신 연구 결과 공유
- 이론 및 실무 측면에서 인공지능, 기계학습의 주요 분야를
논의하고, 학계, 산업계의 연구자와 실무자들에게 해당 분
AIMLA 2024
야의 최첨단 개발 소식 공유
기간 장소 홈페이지' metadata={'source': 'data/SPRI_AI_Brief_2023년12월호_F.pdf', 'file_path': 'data/SPRI_AI_Brief_2023년12월호_F.pdf', 'page': 21, 'total_pages': 23, 'Author': 'dj', 'Creator': 'Hwp 2018 10.0.0.13462', 'Producer': 'Hancom PDF 1.3.0.542', 'CreationDate': "D:20231208132838+09'00'", 'ModDate': "D:20231208132838+09'00'", 'PDFVersion': '1.4'}
page_content='n 참가국들은 튜링상을 수상한 AI 학자인 요슈아 벤지오 교수가 주도하는 ‘과학의 현황(State of
the Science)’ 보고서 작성에도 합의했으며, 보고서를 통해 첨단 AI의 위험과 가능성에 관한
기존 연구를 과학적으로 평가하고 향후 AI 안전 연구를 위한 우선순위를 제시할 계획
n 한국은 영국 정부와 6개월 뒤에 온라인으로 AI 미니 정상회의를 공동 개최하기로 합의했으며,
프랑스 정부와는 1년 후 대면 정상회의를 개최할 예정' metadata={'source':

In [17]:
# 그래프 실행
stream_graph(
    app,
    inputs,
    config,
    ["retrieve", "grade_documents", "query_rewrite", "generate"],
)


==== RETRIEVE ====


==== [CHECK DOCUMENT RELEVANCE TO QUESTION] ====


🔄 Node: grade_documents 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
{"binary_score":"no"}==== [GRADE: DOCUMENT NOT RELEVANT] ====
{"binary_score":"no"}==== [GRADE: DOCUMENT NOT RELEVANT] ====
{"binary_score":"no"}==== [GRADE: DOCUMENT NOT RELEVANT] ====
{"binary_score":"no"}==== [GRADE: DOCUMENT NOT RELEVANT] ====
{"binary_score":"no"}==== [GRADE: DOCUMENT NOT RELEVANT] ====
{"binary_score":"no"}==== [GRADE: DOCUMENT NOT RELEVANT] ====
{"binary_score":"no"}==== [GRADE: DOCUMENT NOT RELEVANT] ====
{"binary_score":"no"}==== [GRADE: DOCUMENT NOT RELEVANT] ====
{"binary_score":"no"}==== [GRADE: DOCUMENT NOT RELEVANT] ====
{"binary_score":"no"}==== [GRADE: DOCUMENT NOT RELEVANT] ====
==== [ASSESS GRADED DOCUMENTS] ====
==== [DECISION: ALL DOCUMENTS ARE NOT RELEVANT TO QUESTION, QUERY REWRITE] ====

==== [REWRITE QUERY] ====


🔄 Node: query_rewrite 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
2024년 노벨 문